In [113]:
import pandas as pd
import requests

FILES_BASE_PATH = '../data/processed/'
file_path = 'api_carrer_info_cleaned.csv'


In [114]:
def create_df_from_csv_file(file_name):
    '''recives an csv path and returns a pandas DF'''
    file_path = FILES_BASE_PATH + file_name
    new_df = pd.read_csv(file_path)
    return new_df

In [115]:
'trameos lista raw y creamos sublistas con el codido de trabajos, segn sean high , medicum, low...'

file_path = '01_FULL_raw_table.csv'
full_raw_df = create_df_from_csv_file(file_path)


In [116]:
def make_sub_by_cat(full_rawdf, text):
        
    reduced_raw = full_raw_df[['uuid','education_level','job_code']]
    reduced_raw = reduced_raw[reduced_raw['job_code'].notna()]
    reduced_raw = reduced_raw[reduced_raw['education_level'].notna()]
    
    
    filter_edu = reduced_raw['education_level'] == text
    list_of_works = reduced_raw[filter_edu]['job_code'].to_list()
    
    
    return list(set(list_of_works))

In [117]:
def api_skill_json (list_of_jobs,text):
    base_url= 'http://api.dataatwork.org/v1/jobs/'
    append_url = '/related_skills'

    json = []


    for i, job_cod in enumerate(list_of_jobs):
        
        i += 1
        url = base_url + job_cod + append_url

        response = requests.get(url)
        job_info = response.json()
        json.append(job_info)
        
        print(f'{text.title()} category jobs examinated for skills: {i}/{len(list_of_jobs)}')
        
        
    return json
    

In [119]:
def create_list_skills_from_json(json):
    list_of_skills  = []
    
    for i in range(len(json)):
        try:
            for e in range(10):
                skill = json[i]['skills'][e]['skill_name']
                list_of_skills.append(skill)

        except:
            pass
        
    return list_of_skills
            
    
    

In [120]:
def created_sorted_dictionary (list_of_skills):
    base_dict = {i:list_of_skills.count(i) for i in list_of_skills}
    ordered_dict = sorted(base_dict.items(), key=lambda x: x[1], reverse=True)
    return ordered_dict

In [128]:
def extract_top_skills(full_raw_df,text):
    
    list_a = make_sub_by_cat(full_raw_df,text)
    
    json = api_skill_json(list_a,text) # capar aquí con api_skill_json(list_a[:X],text)
    
    list_of_skills = create_list_skills_from_json(json)
    
    top_10_skills = created_sorted_dictionary (list_of_skills)[:10]
    
    lista = [f'{a[0].title()} ({a[1]})' for a in top_10_skills]
    
    return lista

In [132]:
# AGREGATED FUNCTION:
def create_df_and_csv(full_raw_df,options):
    'makes a df and a csv file with all the info'
    

    dictionary = {option:extract_top_skills(full_raw_df,option) for option in options}

    df = pd.DataFrame()
    df = pd.DataFrame(dictionary).T.reset_index()

    df.columns = ['Education Level','1st Skill','2nd Skill','3th Skill','4th Skill','5th Sikill',
                  '6th Skill','7th Skill','8th Skill','9th Skill','10th Skill']
    return df

In [133]:
options = ['high','medium','low','no']


In [134]:
create_df_and_csv(full_raw_df,options)

High category jobs examinated for skills: 1/154
High category jobs examinated for skills: 2/154
High category jobs examinated for skills: 3/154
High category jobs examinated for skills: 4/154
High category jobs examinated for skills: 5/154
High category jobs examinated for skills: 6/154
High category jobs examinated for skills: 7/154
High category jobs examinated for skills: 8/154
High category jobs examinated for skills: 9/154
High category jobs examinated for skills: 10/154
High category jobs examinated for skills: 11/154
High category jobs examinated for skills: 12/154
High category jobs examinated for skills: 13/154
High category jobs examinated for skills: 14/154
High category jobs examinated for skills: 15/154
High category jobs examinated for skills: 16/154
High category jobs examinated for skills: 17/154
High category jobs examinated for skills: 18/154
High category jobs examinated for skills: 19/154
High category jobs examinated for skills: 20/154
High category jobs examinated

KeyboardInterrupt: 